In [1]:
from dotenv import load_dotenv
import os

load_dotenv(verbose=True)
key = os.getenv('OPENAI_API_KEY')

In [2]:
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import MessagesPlaceholder
from langchain_community.chat_message_histories import SQLChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory

In [3]:
prompt = ChatPromptTemplate.from_messages(
    [        
        ("system", "You are a helpful assistant."),		    # 시스템 메시지        
        MessagesPlaceholder(variable_name="chat_history"),	# 대화 기록을 위한 Placeholder
        ("human", "{question}"),  				            # 질문
    ]
)

In [5]:
llm = ChatOpenAI(model_name="gpt-4o-mini", temperature=0)

In [4]:
output_parser = StrOutputParser()

In [6]:
chain = prompt | llm | output_parser

In [ ]:
# sqlite.db에서 대화 내용을 가져오는 함수를 만듭니다.
def get_chat_history(user_id, conversation_id):
    return SQLChatMessageHistory(
        table_name=user_id,
        session_id=conversation_id,             
        connection="sqlite:///sqlite.db",       
    )

In [10]:
from langchain_core.runnables.utils import ConfigurableFieldSpec

In [ ]:
# config_fields 를 설정합니다. 이는 대화정보를 조회할 때 참고 정보로 활용합니다.
# user_id: 사용자 ID
# conversation_id: 대화 ID

In [14]:
config_fields = [
    ConfigurableFieldSpec(
        id="user_id",			#  get_chat_history 함수의 파라미터 이름과 같아야 한다.
        annotation=str,
        name="User ID",			
        description="Unique identifier for a user.",
        default="",
        is_shared=True,
    ),
    
    ConfigurableFieldSpec(
        id="conversation_id",	#  get_chat_history 함수의 파라미터 이름과 같아야 한다.
        annotation=str,
        name="Conversation ID",
        description="Unique identifier for a conversation.",
        default="",
        is_shared=True,
    ),
]

In [16]:
chain_with_history = RunnableWithMessageHistory(
    chain,
    
    # 대화 기록을 가져오는 함수를 설정합니다.
    get_chat_history,  
    
    # 입력 메시지의 키를 "question"으로 설정.프롬프트의 question와 맵핑.
    input_messages_key="question", 
    
    # 대화 기록 메시지의 키를 "history"로 설정. MessagesPlaceholder의 chat_history 맵핑.
    history_messages_key="chat_history",
    
    # 대화 기록 조회시 참고할 파라미터를 설정합니다.
    history_factory_config=config_fields,
)

In [17]:
# "configurable" 키 아래에 "user_id", "conversation_id" key-value 쌍을 설정합니다.
config = {
    "configurable": {
        "user_id": "user1", 
        "conversation_id": "conversation1"
    }
}

In [19]:
# 질문에 이름을 물어보는 질문을 해보겠습니다. 이전에 저장한 대화가 있다면, 올바르게 답할 것입니다.

# chain_with_history 객체의 invoke 메서드를 호출하여 질문에 대한 답변을 생성합니다.
# invoke 메서드에는 질문 딕셔너리와 config 설정이 전달됩니다.

# 질문과 config 를 전달하여 실행합니다.
chain_with_history.invoke({"question": "안녕 반가워, 내 이름은 이인환이야"}, config)

'안녕하세요, 이인환님! 다시 만나서 반갑습니다. 어떤 이야기를 나누고 싶으신가요?'

In [20]:
chain_with_history.invoke({"question": "내 이름이 뭐라고?"}, config)

'당신의 이름은 이인환입니다. 맞나요?'